In [ ]:
# 크롤링 하기위한 import선언
import requests
from bs4 import BeautifulSoup
# 크롤링 후 csv파일로 저장하기 위한 import
import csv
import pandas as pd 

# 나중에 페이지가 늘어날 것을 대비하여 만든 것, 마지막 페이지의 수를 int변수로 저장한다.
page_response = requests.get('https://moviekeyword.com')
page_soup = BeautifulSoup(page_response.text, 'html.parser')
last_page = page_soup.select_one('#main > div > nav > div > a:nth-child(8)')
last = last_page.get_text().replace(',','')
int_last = int(last)

# start_url뒤에 페이지 수를 추가해주면 그 페이지로 이동한다.
start_url = 'https://moviekeyword.com/page/'

def page_jump(n):
    movies_data = []
    for i in range(50*(n) + 1, 50*(n+1)+1): #뒤의 if문과 연관시켜서 값을 넣으면 된다.
        page_num = i
        URL = start_url + str(page_num)
        response = requests.get(URL)

        soup = BeautifulSoup(response.text, 'html.parser')

        # 여기까지 페이지 관련으로 for문을 받아 한바퀴를 돌때마다, 다음 페이지로 이동

        # title 추출 시작
        movies_page_list = soup.select('div[id=page] > div[id=content-area] > div[id=content] > div[id=primary] > main[id=main] > div[id=primary] > main[id=main] > article')

        for movie_page in movies_page_list:
            a_tag = movie_page.select_one('div.post-summary-container.col-md-11 > header > h2 > a')
            movies_title = a_tag.get_text() # title 추출
            movies_link = a_tag['href'] # 각 페이지에 접속하여 content와 keyword를 추출하기 위하여 주소 추출

            # 여기까지 title 추출하기

            # 추출한 링크로 이동하기 위한 response를  mv_response로 설정
            mv_response = requests.get(movies_link)
            new_soup = BeautifulSoup(mv_response.text, 'html.parser')

            # 중간에 content가 5번 위치일 경우 사용할 경로
            movie_content = new_soup.select_one('#right-sidebar > div.right-sidebar-meta-container > ul > li:nth-child(5) > div.meta-value-container > span.meta-value > a')
            # 5번 위치가 아니라 4번에 있는 경우 사용할 경로
            movie_content_reserve = new_soup.select_one('#right-sidebar > div.right-sidebar-meta-container > ul > li:nth-child(4) > div.meta-value-container > span.meta-value > a')
            
            #  위에서 설명한데로 5번이 비어있는경우 4번에서 값을 가지고 온다는 if문
            if movie_content != None:
                content = movie_content.get_text()
            elif movie_content_reserve != None:
                content = movie_content_reserve.get_text()
            else:
                content = '없음'
            
            # 여기까지 장르(content) 추출하기

            movie_keywords = new_soup.select('#right-sidebar > div.right-sidebar-meta-container > ul > li:nth-last-child(1) > div.meta-value-container > span.meta-value')
            
            keywords = []
            # for 문으로 keyword 값들을 각각 가지고 와서 keywords에 append시킨다.
            for movie_keword in movie_keywords:
                keyword_tag = movie_keword.select_one('a')
                # if문은 keyword가 없는 경우 키워드가 없다는 것을 출력시키기 위한 것
                if keyword_tag != None: 
                    keyword = keyword_tag.get_text().strip()
                else:
                    keyword = '없음'
                keywords.append(keyword)

            # 여기까지 키워드 추출하기

            movie_data = {
                'title': movies_title,
                'content': content,
                'keyword': keywords
            }
            movies_data.append(movie_data)
            # print(movie_data) # 영화 제목, 장르, 키워드 출력
            # print(movies_title) #영화 제목 출력
            # print(content) # 영화 장르 출력
            # print(keywords) # 영화 키워드 출력
            print(i, " ", movie_data) 
            # 에러값의 원인을 찾기위해 페이지 값과 영화데이터 값을 각각 출력
            # 일단 출력이 되었으면 거기까지는 문제가 없다는 뜻이다.

        # 위의 첫 for문과 연계 너무 많이하면 오류발생되서 데이터가 하나도 저장이 안됨
        # if문을 사용하여 20페이지 당 파일 이름을 바꾸어서 데이터를 새로 저장
        if i % 50 == 0 or i == int_last:
            dataframe = pd.DataFrame(movies_data)
            dataframe.to_csv(f"./save_data_{n}.csv",encoding='UTF-16',header=False,index=False)

n = 0 # 해당 폴더에서 출력되지 않은 숫자를 입력하면 그것부터 다시 출력시킨다.
       
for a in range(n+1,(int_last//50)+2):
    page_jump(n)
    n += 1